In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

%matplotlib inline
print(tf.__version__)
sess = tf.InteractiveSession()
tf.logging.set_verbosity("INFO")

/home/filbert/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


1.4.0


In [2]:
# define preprocessing function
def preprocess(disease, first, last, data, label):
    for count in range(first,last+1): #only taking first 48 pics, last 5 to act as a '''test set'''
        string = "processed/" + str(disease) + "/" + str(count) + ".jpg"
        pic = Image.open(string).load()
        rgb = []
        for x in range(0,64):
            for y in range(0,64):
                rgb.append(pic[x,y])

        data.append(rgb)
        if disease == "acne":
            label.append(0)
        elif disease == "ezcema":
            label.append(1)

def onehot(array,depth):
    n = array.size
    onehot = np.zeros((n,depth))
    for count in range(0,array.size):
        onehot[count][array[count]] = 1
        
    return onehot



# define training data
train_data = []
train_labels = []

preprocess("acne",1,48,train_data,train_labels)
preprocess("ezcema",1,51,train_data,train_labels)
    
train_data = np.asarray(train_data, dtype=np.float32)
train_labels = np.asarray(train_labels)

# define testing data
test_data = []
test_labels = []

preprocess("acne",49,53,test_data,test_labels)
preprocess("ezcema",52,56,test_data,test_labels)

test_data = np.asarray(test_data, dtype=np.float32)
test_labels = np.asarray(test_labels)

# reshape data
train_labels = onehot(train_labels,2)
test_labels = onehot(test_labels,2)

train_data = train_data.reshape([-1, 64, 64, 3])
test_data = test_data.reshape([-1, 64, 64, 3])

In [3]:
test_labels.shape

(10, 2)

In [4]:
# define network architecture
# input
network = input_data(shape=[None, 64, 64, 3], name='input')

# conv 1 and pooling 1
network = conv_2d(network, 64, 13, activation = "relu", regularizer="L2")
network = max_pool_2d(network, 2, 2)
network = local_response_normalization(network)

# conv 2 and pooling 2
network = conv_2d(network, 96, 13, activation = "relu", regularizer="L2")
network = max_pool_2d(network, 2, 2)
network = local_response_normalization(network)

# dense and dropout
network = fully_connected(network, 1024, activation = "tanh")
network = dropout(network, 0.9)

# logits
network = fully_connected(network, 2, activation = "softmax")

# target
network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

# training
dermacam = tflearn.DNN(network, tensorboard_verbose=0)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [5]:
dermacam.fit({'input': train_data}, {'target': train_labels}, n_epoch=10,
           validation_set=({'input': test_data}, {'target': test_labels}),
           snapshot_step=10, show_metric=True, run_id='convnet_dermacam')

Training Step: 19  | total loss: 2.79979 | time: 15.179s
| Adam | epoch: 010 | loss: 2.79979 - acc: 0.4812 -- iter: 64/99
Training Step: 20  | total loss: 3.01296 | time: 24.538s
| Adam | epoch: 010 | loss: 3.01296 - acc: 0.4822 | val_loss: 5.73860 - val_acc: 0.5000 -- iter: 99/99
--
